# V1 取得 VE2 Agent 3.

In [1]:
from ckipnlp.pipeline import CkipPipeline, CkipDocument, CkipCorefPipeline
import pandas as pd
from tqdm import trange
import ast

## 載檔

In [2]:
file_full_name = "./v1_Total_Opinion.csv"

news_df = pd.read_csv(file_full_name)
all_output_file = "v1_Total_Opinion.csv"
opinion_after_cluster_file = "v1_cluster_opinions.csv"
topic_info_file = "v1_topic_info.csv"

In [3]:
news_df.head(1)

,id,article,category,company_id,reporter,timestamp,title,url_pattern,label,annotator,annotation_id,created_at,updated_at,lead_time,article_coref,people_name,ner
0,883,"針對行政院長蘇貞昌、國民黨立委鄭麗文12日在立法院發生激烈爭執,兼任民進黨主席的總統蔡英文1...",政治,3,陶本和,1634110500,蔡英文發聲挺蘇貞昌 批鄭麗文情緒性發言對當事人不公,2100273,"[{""start"": 0, ""end"": 142, ""text"": ""針對行政院長蘇貞昌、國...",1,37,2022-02-03T14:55:55.310663Z,2022-02-03T15:02:18.598120Z,518.272,"針對行政院長蘇貞昌、國民黨立委鄭麗文12日在立法院發生激烈爭執,兼任民進黨主席的總統蔡英文1...","['蔡英文', '行政院長', '鄭麗文', '民進黨', '蘇貞昌', '台灣人民', '...","['蔡英文', '行政院長', '12日', '鄭麗文', '台灣', '民進黨', '蘇貞..."


#### 輸入 ckip 帳號密碼

In [4]:
pipeline = CkipPipeline(opts={'con_parser': {'username': '', 'password': ''}})
coref_pipeline = CkipCorefPipeline(opts={'con_parser': {'username': '', 'password': ''}})

## 3. 產生 VE2 agent pair


### function 定義

In [5]:
def make_conparse(an_article, pipeline):
    doc = CkipDocument(raw=an_article)
    pipeline.get_conparse(doc)
    return doc.conparse[0]

In [6]:
def find_spice_index(an_article):
    return [i for i, char in enumerate(an_article) if char == ' ']

In [7]:
def make_agent_VE2_label(old_article, pipeline):

  spice_index_list = find_spice_index(old_article)
  conparse = make_conparse(old_article, pipeline)
  # print_tree_pic(old_article)

  article = ""
  start = 0
  end = 0
  # every sentence in article have 4 kinds VE2 and agent pairs
  one_to_one_pair = []
  # iterate the clause
  for ParseClause in conparse:
    sentence = ""
    if ParseClause.clause != '':
      VE2_count = 0
      agent_count = 0
      VE2_agent_clause_label = []
      tree = ParseClause.to_tree()
      # iterate every node in sentence to find VE2 and agent
      for node in tree.all_nodes_itr():
        node_dict = node.to_dict()
        node_data = node_dict['data']

        node_id = node_dict['id']
        role = node_data['role']
        pos = node_data['pos']
        word = node_data['word']
        # change index when switch words
        if word != None:
          sentence += word
          start = end
          end = start + len(word)
        else:
          pass
        # find VE2
        if 'VE2' in pos:
          label = {'start': start, 'end': end, 'text': word, 'labels': ['VE2']}

          VE2_agent_clause_label.append(label.copy())
          VE2_count += 1
        # find agent
        if role != None:
          if 'agent' in role:
            if word == None:
              temp_word = ""
              sub_tree = tree.subtree(node_id)
              sub_node_list = sub_tree.all_nodes()
              sub_node_list.sort(key=lambda x: x.to_dict()['id'])
              for sub_node in sub_node_list:
                sub_node_dict = sub_node.to_dict()
                if sub_node_dict['data']['word'] != None:
                  temp_word += sub_node_dict['data']['word']
              temp_start = end
              temp_end = end + len(temp_word)
            else:
              temp_word = word
              temp_start = start
              temp_end = end
            label = {'start': temp_start, 'end': temp_end, 'text': temp_word, 'labels': ['agent']}
            VE2_agent_clause_label.append(label.copy())
            agent_count += 1

    # change index when switch sentence
    sentence += ParseClause.delim
    # print(ParseClause.delim)
    start = end
    end = start + len(ParseClause.delim)
    article += sentence

    # check this sentence's VE2_agent pair belong to which kind
    if VE2_count == 1 and agent_count == 1:
      one_to_one_pair.append(VE2_agent_clause_label.copy())

  # change index when article have space
  for space_index in spice_index_list:
    article = article[:space_index] + " " + article[space_index:]
    for one_to_one_sentence_object in one_to_one_pair:
      for one_to_one_object in one_to_one_sentence_object:
        if one_to_one_object['start'] >= space_index:
          one_to_one_object['start'] += 1
          one_to_one_object['end'] += 1        
    
  return one_to_one_pair

## 6. 開始實作

### 3.

In [8]:
article_list = news_df['article_coref'].tolist()

In [9]:
news_df['one_to_one_pair'] = ""
news_df['one_to_one_pair_count'] = 0

for i in trange(len(news_df)):
    if i == 5:
        break
    try:
        news_df['one_to_one_pair'][i] = make_agent_VE2_label(article_list[i], pipeline)
    except:
        print('function return error in' + str(i))
        continue
    try:
        news_df['one_to_one_pair_count'][i] = len(news_df['one_to_one_pair'][i])
    except:
        print('one_to_one_pair_count error in' + str(i))
        continue
    

  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
C:\Users\User\AppData\Local\Temp/ipykernel_16856/3628932566.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [11]:
news_df.head(5)

,id,article,category,company_id,reporter,timestamp,title,url_pattern,label,annotator,annotation_id,created_at,updated_at,lead_time,article_coref,people_name,ner,one_to_one_pair,one_to_one_pair_count
0,883,"針對行政院長蘇貞昌、國民黨立委鄭麗文12日在立法院發生激烈爭執,兼任民進黨主席的總統蔡英文1...",政治,3,陶本和,1634110500,蔡英文發聲挺蘇貞昌 批鄭麗文情緒性發言對當事人不公,2100273,"[{""start"": 0, ""end"": 142, ""text"": ""針對行政院長蘇貞昌、國...",1,37,2022-02-03T14:55:55.310663Z,2022-02-03T15:02:18.598120Z,518.272,"針對行政院長蘇貞昌、國民黨立委鄭麗文12日在立法院發生激烈爭執,兼任民進黨主席的總統蔡英文1...","['蔡英文', '行政院長', '鄭麗文', '民進黨', '蘇貞昌', '台灣人民', '...","['蔡英文', '行政院長', '12日', '鄭麗文', '台灣', '民進黨', '蘇貞...","[[{'start': 134, 'end': 138, 'text': '台灣人民', '...",9
1,862,"行政院長蘇貞昌12日在立法院總質詢時,怒斥國民黨立委鄭麗文「袂見笑」、「當時還要拜託我去幫你...",政治,3,呂晏慈,1634092560,蘇貞昌酸鄭麗文「民進黨叛徒」 他反譏:蔡英文也當國民黨閣員,2100056,"[{""start"": 0, ""end"": 51, ""text"": ""行政院長蘇貞昌12日在立...",1,16,2022-02-03T12:47:32.966394Z,2022-02-03T12:47:43.626597Z,30432.838,"行政院長蘇貞昌12日在立法院總質詢時,怒斥國民黨立委鄭麗文「袂見笑」、「當時還要拜託我去幫你...","['袂見', '蘇貞昌', '民進黨', '鄭麗文', '李德維', '陳明文', '國民黨...","['袂見', '蘇貞昌', '蘇', '費鴻泰酸', '蘇貞昌', '鄭先', '民進黨',...","[[{'start': 86, 'end': 94, 'text': '黨團總召費鴻泰酸',...",7
2,854,"台灣基進立委陳柏惟遭罷免團體掛看板質疑「反對採購疫苗」,立法院副院長蔡其昌4日反駁,陳柏惟反...",政治,3,呂晏慈,1634090220,蔡其昌質疑「想買中國科興疫苗」 國民黨團今按鈴控告妨害名譽,2100001,"[{""start"": 0, ""end"": 27, ""text"": ""台灣基進立委陳柏惟遭罷免...",1,8,2022-02-03T05:22:16.944255Z,2022-02-03T06:27:27.039925Z,7616.256,"台灣基進立委陳柏惟遭罷免團體掛看板質疑「反對採購疫苗」,立法院副院長蔡其昌4日反駁,陳柏惟反...","['國民黨團', 'BNT', '蔡其昌', '中國科興', '陳柏惟', '莫德納', '...","['上午', '國民黨團', '4日', '中國', 'BNT', '蔡其昌', '國民黨團...","[[{'start': 0, 'end': 9, 'text': '台灣基進立委陳柏惟', ...",3
3,953,"實況主丁特日前因質疑「天堂M」遊戲道具製作機率公告不實,引起各界關注。民眾黨立委高虹安14日...",政治,3,張鈞量,1634184240,丁特控「天堂M」製作機率造假 高虹安揪朝野立委推「轉蛋法」修法,2101034,"[{""start"": 0, ""end"": 35, ""text"": ""實況主丁特日前因質疑「天...",1,107,2022-02-04T13:48:13.063893Z,2022-02-04T13:48:13.063922Z,263.836,"實況主丁特日前因質疑「天堂M」遊戲道具製作機率公告不實,引起各界關注。民眾黨立委高虹安14日...","['時代力量', '高虹安', '虹安', '紫布', 'Microsoft', '洪孟楷'...","['時代力量', '高虹安', '虹安', '2018年', '高虹安', '上百萬', '...","[[{'start': 15, 'end': 19, 'text': '遊戲道具', 'la...",9
4,952,"國防部長邱國正14日證實,長年從事後備動員業務規劃的國防部全民防衛動員綜合作業室主任韓岡明,...",政治,3,林銘翰,1634182920,全民防衛動員署明年成立 全動室主任韓岡明屆齡退休,2101023,"[{""start"": 0, ""end"": 106, ""text"": ""國防部長邱國正14日證...",1,106,2022-02-04T13:43:48.981022Z,2022-02-04T13:43:48.981050Z,174.457,"國防部長邱國正14日證實,長年從事後備動員業務規劃的國防部全民防衛動員綜合作業室主任韓岡明,...","['全民防衛動員署', '陳以信', '國防部', '邱國正', '韓岡明', '國民黨',...","['全民防衛動員署', '陳以信', '一半', '國防委員會', '國防部', '邱國正'...","[[{'start': 0, 'end': 7, 'text': '國防部長邱國正', 'l...",8


In [12]:
news_df.to_csv('./v1_Total_Opinion.csv', index=False)